In [7]:
import numpy as np
from scipy.sparse import rand as sprand
import torch

# Make up some random explicit feedback ratings
# and convert to a numpy array
n_users = 1_000
n_items = 1_000
ratings = sprand(n_users, n_items, density=0.01, format="csr")
ratings.data = np.random.randint(1, 5, size=ratings.nnz).astype(np.float64)
ratings = ratings.toarray()

In [8]:
ratings

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 3.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
class MatrixFactorization(torch.nn.Module):
    def __init__(self, n_users, n_items, n_factors=20):
        super().__init__()
        self.user_factors = torch.nn.Embedding(n_users, n_factors, sparse=True)
        self.item_factors = torch.nn.Embedding(n_items, n_factors, sparse=True)

    def forward(self, user, item):
        return (self.user_factors(user) * self.item_factors(item)).sum(1)

In [22]:
class BiasedMatrixFactorization(torch.nn.Module):
    def __init__(self, n_users, n_items, n_factors=20):
        super().__init__()
        self.user_factors = torch.nn.Embedding(n_users, n_factors, sparse=True)
        self.item_factors = torch.nn.Embedding(n_items, n_factors, sparse=True)
        self.user_biases = torch.nn.Embedding(n_users, 1, sparse=True)
        self.item_biases = torch.nn.Embedding(n_items, 1, sparse=True)

    def forward(self, user, item):
        pred = self.user_biases(user) + self.item_biases(item)
        pred += (
            (self.user_factors(user) * self.item_factors(item))
            .sum(dim=1, keepdim=True)
        )
        return pred.squeeze()

In [29]:
model = MatrixFactorization(n_users, n_items, n_factors=32)

In [30]:
loss_func = torch.nn.MSELoss()

In [31]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)  # learning rate
adagrad_loss = torch.optim.Adagrad(model.parameters(), lr= 0.0001)

In [32]:
# Sort our data
rows, cols = ratings.nonzero()
p = np.random.permutation(len(rows))
rows, cols = rows[p], cols[p]

num_epochs = 50
for epoch in range(num_epochs):
    for row, col in zip(*(rows, cols)):
        # Set gradients to zero
        adagrad_loss.zero_grad()
        
        # Turn data into tensors
        rating = torch.FloatTensor([ratings[row, col]])
        row = torch.LongTensor([row])
        col = torch.LongTensor([col])

        # Predict and calculate loss
        prediction = model(row, col)
        loss = loss_func(prediction, rating)

        # Backpropagate
        loss.backward()

        # Update the parameters
        adagrad_loss.step()
    
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

Epoch 1/50, Loss: 1.2482445240020752
Epoch 2/50, Loss: 1.247576355934143
Epoch 3/50, Loss: 1.247113823890686
Epoch 4/50, Loss: 1.2467371225357056
Epoch 5/50, Loss: 1.2464128732681274
Epoch 6/50, Loss: 1.2461233139038086
Epoch 7/50, Loss: 1.2458581924438477
Epoch 8/50, Loss: 1.2456133365631104
Epoch 9/50, Loss: 1.2453835010528564
Epoch 10/50, Loss: 1.2451679706573486
Epoch 11/50, Loss: 1.2449649572372437
Epoch 12/50, Loss: 1.2447694540023804
Epoch 13/50, Loss: 1.244583249092102
Epoch 14/50, Loss: 1.2444047927856445
Epoch 15/50, Loss: 1.2442338466644287
Epoch 16/50, Loss: 1.2440694570541382
Epoch 17/50, Loss: 1.2439078092575073
Epoch 18/50, Loss: 1.2437500953674316
Epoch 19/50, Loss: 1.2435990571975708
Epoch 20/50, Loss: 1.243451476097107
Epoch 21/50, Loss: 1.2433061599731445
Epoch 22/50, Loss: 1.2431657314300537
Epoch 23/50, Loss: 1.2430282831192017
Epoch 24/50, Loss: 1.2428942918777466
Epoch 25/50, Loss: 1.2427630424499512
Epoch 26/50, Loss: 1.2426344156265259
Epoch 27/50, Loss: 1.2425

KeyboardInterrupt: 